***
### Import of required libraries
***

In [ ]:
import pandas as pd
from traffic.core import Traffic

***
### Import of data
***

In [ ]:
# SAMAX data
samax = Traffic.from_file(
    "/mnt/beegfs/store/krum/MT/inputs/samax_trajs.parquet"
)

# Mass data
mass = pd.read_parquet("/mnt/beegfs/store/krum/MT/inputs/df_mass.parquet")

# T/RH/P data
trhp = pd.read_parquet("/mnt/beegfs/store/krum/MT/inputs/df_t_rh_p.parquet")

# Wind data
wind = pd.read_parquet("/mnt/beegfs/store/krum/MT/inputs/df_wind.parquet")

# Flightplan data
fp = pd.read_parquet("/mnt/beegfs/store/krum/MT/inputs/df_sid.parquet")

***
### Merge of data
***

##### mass/typecode data

In [ ]:
# FZAG mass/typecode data
samax.data["date"] = pd.to_datetime(samax.data["timestamp"].dt.date)
samax.data = pd.merge(
    left=samax.data,
    right=mass,
    left_on=["date", "callsign"],
    right_on=["date", "callsign"],
    how="left",
)

##### T/RH/P data

In [ ]:
samax.data = pd.merge_asof(
    left=samax.data.sort_values("timestamp"),
    right=trhp.sort_values("timestamp"),
    on="timestamp",
    tolerance=pd.Timedelta("30min"),
)

##### Wind data

In [ ]:
samax.data = pd.merge_asof(
    left=samax.data.sort_values("timestamp"),
    right=wind.sort_values("timestamp"),
    on="timestamp",
    tolerance=pd.Timedelta("30min"),
)

##### Flightplan data

In [ ]:
samax.data = pd.merge(
    left=samax.data,
    right=fp,
    left_on=["flight_id"],
    right_on=["flight_id"],
    how="left",
)

***
### Keep only flights with complete data and save to file
***

In [ ]:
# Drop flights with missing values
delete_ids = (
    samax.data[
        samax.data[
            [
                "timestamp",
                "latitude",
                "longitude",
                "altitude",
                "typecode",
                "toff_weight_kg",
                "temperature_gnd",
                "humidity_gnd",
                "pressure_gnd",
                "wind_x_2min_avg",
                "wind_y_2min_avg",
                "hour",
                "weekday",
                "month",
                "SID",
            ]
        ]
        .isna()
        .any(axis=1)
    ]["flight_id"]
    .unique()
    .tolist()
)
samax.data = samax.data[~samax.data["flight_id"].isin(delete_ids)]

# Save traffic as parquet
samax.to_parquet("/mnt/beegfs/store/krum/MT/traffic_complete/complete.parquet")